In [105]:
import numpy as np

# read file 

data = open("my_input/data.txt", "r")
param = open("my_input/parameters.txt", "r")



In [106]:
# read lines from file as numbers   

rainfall = []
for line in data:
    rainfall.append(float(line))


N = int(param.readline())

trans_matrix = []

for i in range(N):
    # read line from param
    p = [float(x) for x in param.readline().split()]
    trans_matrix.append(p)


# read distro

means = [float(x) for x in param.readline().split()]
sds = [float(x) for x in param.readline().split()]





In [107]:
# gaussian distribution function

def gauss(x, mu, sigma):
    return (1.0 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)

# find stationary distribution for given transition matrix

def get_stationary_distribution(A,pi):
    A = np.array(A)
    pi = np.array(pi)
    
    N = len(A)
    
    # initial state
    x = pi
    
    # recursion
    for i in range(N):
        x = np.dot(x,A)
    
    return x


In [108]:

# emission probabilities
emisson_matrix = []

n = len(trans_matrix)

for i in range(n):
    prob = []
    for d in rainfall:
        prob.append(gauss(d, means[i], sds[i]))
    emisson_matrix.append(prob)

        

In [109]:
# viterbi algorithm

def viterbi(A,B,pi) :
    A = np.array(A)
    B = np.array(B)
    pi = np.array(pi)
 
    N = len(A)
    T = len(B[0])
    
    # initialize
    delta = np.zeros((N,T))
    psi = np.zeros((N,T))

    # initial state
    delta[:,0] = pi*B[:,0]

    # recursion
    for t in range(1,T):
        for j in range(N):
            delta[j,t] = np.max(delta[:,t-1]*A[:,j]) * B[j,t]   
    
    # psi
    for t in range(1,T):
        for j in range(N):
            psi[j,t] = np.argmax(delta[:,t-1]*A[:,j])

    
    sol = np.zeros(T)

    sol[T-1] = np.argmax(delta[:,T-1])
    
    for t in range(T-2,-1,-1):
        sol[t] = psi[int(sol[t+1]),t+1]
    
    return sol



pi = get_stationary_distribution(trans_matrix,[0.5,0.5])

Sol =  viterbi(trans_matrix,emisson_matrix,pi)

# solution write in output file

out = open("my_output/wo_learing.txt","w")

for s in Sol :
    if s == 1 :
        out.write("La Nina\n")
    else:
        out.write("El Nino\n")



    

In [110]:
# forward algorithm
def alpha_func(theta):

    A,B,pi = theta
    A = np.array(A)
    B = np.array(B)
    pi = np.array(pi)

    T = len(B[0])
    
    N = len(A)
    alpha = np.zeros((N,T))
    
    # initial state
    alpha[:,0] = pi*B[:,0]
    
    # recursion
    for t in range(1,T):
        for j in range(N):
            alpha[j,t] = np.sum(alpha[:,t-1]*A[:,j]) * B[j,t]
    
    return alpha

# backward algorithm

def beta_func(theta):

    A,B,pi = theta
    A = np.array(A)
    B = np.array(B)
    pi = np.array(pi)

    
    
    N = len(A)
    T = len(B[0])

    beta = np.zeros((N,T))
    
    # initial state
    beta[:,T-1] = 1
    
    # recursion
    for t in range(T-2,-1,-1):
        for j in range(N):
            beta[j,t] = np.sum(A[:,j]*B[j,t+1]*beta[:,t+1])
    
    return beta


# gamma function

def gamma_func(alpha,beta):

    N = len(alpha)
    T = len(alpha[0])
    gamma = np.zeros((N,T))
    
    for t in range(T):
        sum = 0
        for j in range(N):
            sum += alpha[j,t]*beta[j,t]
        for j in range(N):
            gamma[j,t] = alpha[j,t]*beta[j,t]/sum
    
    return gamma


def zeye_func(alpha,beta,theta) :
    A,B,pi = theta
    A = np.array(A)
    B = np.array(B)
    pi = np.array(pi)

    N = len(A)
    T = len(B[0])

    zeye = np.zeros((N,N,T))

    for t in range(T-1):
        sum = 0
        for i in range(N):
            for j in range(N):
                sum += alpha[i,t]*A[i,j]*B[j,t+1]*beta[j,t+1]
        
        for i in range(N):
            for j in range(N):
                zeye[i,j,t] = alpha[i,t]*A[i,j]*B[j,t+1]*beta[j,t+1]/sum
    
    return zeye



def Baum_Welch_iter(theta) :

    A,B,pi = theta
    A = np.array(A)
    B = np.array(B)
    pi = np.array(pi)

    N = len(A)
    T = len(B[0])

    alpha = alpha_func(theta)
    #print(alpha)
    beta = beta_func(theta)
    #print(beta)
    gamma = gamma_func(alpha,beta)
    #print(gamma)
    zeye = zeye_func(alpha,beta,theta)

    # update A
    for i in range(N):
        for j in range(N):
            sum_ij = 0
            sum_i = 0
            for t in range(T-1):
                sum_ij += zeye[i,j,t]
                sum_i += gamma[i,t]
            A[i,j] = sum_ij/sum_i

    # update B
    for j in range(N):
        for k in range(T):
            sum_jk = 0
            sum_j = 0

            for t in range(T):
                if t == k :
                    sum_jk += gamma[j,t]
                sum_j += gamma[j,t]
            
            B[j,k] = sum_jk/sum_j

    # update pi
    sum = 0
    for i in range(N):
        sum += gamma[i,0]
    pi = np.zeros(N)
    for i in range(N):
        pi[i] = gamma[i,0]/sum


    return [A,B,pi]
    


def Rum_algo(theta,iter_num) :
    A,B,pi = theta
    A = np.array(A)
    B = np.array(B)
    pi = np.array(pi)

    N = len(A)
    T = len(B[0])

    for i in range(iter_num):
        theta = Baum_Welch_iter(theta)
        print("run ",i,theta)

theta = [trans_matrix,emisson_matrix,pi]

Rum_algo(theta,5)

run  0 [array([[0.67235102, 0.21954693],
       [0.20211149, 0.84025586]]), array([[1.18623015e-44, 9.22196525e-47, 2.12140732e-47, 5.10974091e-48,
        2.16930124e-45, 3.23093878e-23, 2.68163228e-01, 3.10129258e-01,
        2.63425193e-01, 1.58282321e-01],
       [1.43074671e-01, 1.43074671e-01, 1.43074671e-01, 1.43074671e-01,
        1.43074671e-01, 1.43074671e-01, 2.75642433e-02, 9.48751370e-03,
        2.96051362e-02, 7.48950821e-02]]), array([3.57131509e-44, 1.00000000e+00])]
run  1 [array([[0.90763905, 0.05090202],
       [0.18651035, 0.83390681]]), array([[1.20169741e-133, 1.44744318e-093, 8.02004717e-095,
        8.20111163e-093, 5.88095833e-067, 4.04575320e-022,
        2.60278923e-001, 2.61161110e-001, 2.55944034e-001,
        2.22615934e-001],
       [1.61815998e-001, 1.61815998e-001, 1.61815998e-001,
        1.61815998e-001, 1.61815998e-001, 1.61815998e-001,
        9.21983595e-004, 3.76650728e-004, 3.60163847e-003,
        2.42037362e-002]]), array([4.59065396e-133, 1.0